In [ ]:
# Local use: load Santiment API key from a local env file
import os
from pathlib import Path

env_path = Path("santiment.env")
if env_path.exists():
    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        key, _, value = line.partition("=")
        os.environ[key.strip()] = value.strip()

API_KEY = os.environ.get("SANTIMENT_API_KEY")

print(API_KEY)



In [ ]:
# Colab use: retrieve Santiment API key from Colab userdata
from google.colab import userdata

API_KEY = userdata.get("SANTIMENT_API_KEY")



In [3]:
"""
Santiment API Client
Retrieve social volume, weighted sentiment, exchange flows, addresses, MVRV, and NPL data
"""

import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union
import logging
import pytz
from pydantic import BaseModel

try:
    API_KEY  # type: ignore  # set by local or Colab cell
except NameError:
    API_KEY = os.environ.get('SANTIMENT_API_KEY')

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class SantimentData(BaseModel):
    """Data model for Santiment metrics"""
    slug: str
    metric: str
    datetime: datetime
    value: Union[float, int, None]
    metadata: Optional[Dict] = {}

class SantimentClient:
    """Client for Santiment API"""
    
    def __init__(self, api_key: str):
        """
        Initialize Santiment client
        
        Args:
            api_key: Santiment API key
        """
        self.api_key = api_key
        self.base_url = "https://api.santiment.net/graphql"
        self.session = requests.Session()
        self.session.headers.update({
            'Authorization': f'Apikey {api_key}',
            'Content-Type': 'application/json'
        })
        
        # Rate limit tracking
        self.rate_limit_info = {
            'limit': None,                    # General limit
            'remaining': None,                # General remaining
            'limit_minute': None,
            'remaining_minute': None,
            'limit_hour': None,
            'remaining_hour': None,
            'limit_month': None,
            'remaining_month': None,
            'reset': None,
            'last_updated': None
        }
        
        # Available metrics
        self.metrics = {
            'social_volume': 'social_volume_total',
            'weighted_sentiment': 'sentiment_balance_total',
            'exchange_inflow': 'exchange_inflow',
            'exchange_outflow': 'exchange_outflow',
            'active_addresses': 'active_addresses_24h',
            'mvrv_usd': 'mvrv_usd_30d',
            'npl': 'network_profit_loss',
            'marketcap_usd': 'marketcap_usd',
            'price_usd': 'price_usd'
        }
    
    def query_graphql(self, query: str) -> Dict:
        """
        Execute GraphQL query and extract rate limit information
        
        Returns:
            Dict containing the GraphQL response data
        """
        try:
            response = self.session.post(
                self.base_url,
                json={'query': query},
                timeout=30
            )
            response.raise_for_status()
            
            # Store response for header inspection
            self.last_response = response
            
            # Extract rate limit headers
            self._update_rate_limit_info(response.headers)
            
            return response.json()
        except requests.exceptions.RequestException as e:
            logger.error(f"GraphQL query failed: {e}")
            raise
    
    def _update_rate_limit_info(self, headers: Dict):
        """Update rate limit information from response headers"""
        # Parse exactly the same way - use direct key access like limit_minute
        self.rate_limit_info['limit'] = headers.get('x-ratelimit-limit')
        self.rate_limit_info['remaining'] = headers.get('x-ratelimit-remaining')
        self.rate_limit_info['limit_minute'] = headers.get('x-ratelimit-limit-minute')
        self.rate_limit_info['remaining_minute'] = headers.get('x-ratelimit-remaining-minute')
        self.rate_limit_info['limit_hour'] = headers.get('x-ratelimit-limit-hour')
        self.rate_limit_info['remaining_hour'] = headers.get('x-ratelimit-remaining-hour')
        self.rate_limit_info['limit_month'] = headers.get('x-ratelimit-limit-month')
        self.rate_limit_info['remaining_month'] = headers.get('x-ratelimit-remaining-month')
        self.rate_limit_info['reset'] = headers.get('x-ratelimit-reset')
        self.rate_limit_info['last_updated'] = datetime.now()
        
        # Log if rate limit is low
        if self.rate_limit_info['remaining_minute']:
            remaining = int(self.rate_limit_info['remaining_minute'])
            if remaining < 3:
                logger.warning(f"Rate limit low: {remaining} calls remaining")
    
    def get_rate_limit_status(self) -> Dict:
        """
        Get current rate limit status
        
        Returns:
            Dict with rate limit information including ALL fields from rate_limit_info
        """
        # Return a copy with ALL rate limit fields - don't filter anything
        info = dict(self.rate_limit_info)  # Make sure we get all fields
        
        # Calculate usage percentages if we have both limit and remaining (optional calculations)
        if info.get('limit_minute') and info.get('remaining_minute'):
            try:
                limit = int(info['limit_minute'])
                remaining = int(info['remaining_minute'])
                used = limit - remaining
                info['used_minute'] = used
                info['usage_percent_minute'] = (used / limit * 100) if limit > 0 else 0
            except (ValueError, TypeError):
                pass
        
        if info.get('limit_month') and info.get('remaining_month'):
            try:
                limit_month = int(info['limit_month'])
                remaining_month = int(info['remaining_month'])
                used_month = limit_month - remaining_month
                info['used_month'] = used_month
                info['usage_percent_month'] = (used_month / limit_month * 100) if limit_month > 0 else 0
            except (ValueError, TypeError):
                pass
        
        if info.get('reset'):
            try:
                reset_seconds = int(info['reset'])
                reset_time = datetime.now() + timedelta(seconds=reset_seconds)
                info['reset_time'] = reset_time
            except (ValueError, TypeError):
                pass
        
        return info
    
    def can_make_request(self, min_remaining: int = 1) -> bool:
        """
        Check if it's safe to make an API request
        
        Args:
            min_remaining: Minimum number of calls that should remain (default: 1)
        
        Returns:
            True if safe to make request, False otherwise
        """
        status = self.get_rate_limit_status()
        if status['remaining_minute']:
            return int(status['remaining_minute']) >= min_remaining
        return True  # If we don't have rate limit info, assume it's safe
    
    def wait_for_rate_limit(self, min_remaining: int = 5):
        """
        Wait if rate limit is too low
        
        Args:
            min_remaining: Minimum number of calls that should remain before proceeding
        """
        status = self.get_rate_limit_status()
        if status['remaining_minute'] and int(status['remaining_minute']) < min_remaining:
            if status['reset']:
                reset_seconds = int(status['reset'])
                logger.info(f"Rate limit low. Waiting {reset_seconds} seconds for reset...")
                time.sleep(reset_seconds + 1)  # Wait a bit extra to be safe
            else:
                logger.warning("Rate limit low but no reset time available. Waiting 60 seconds...")
                time.sleep(60)
    
    def get_metric_data(
        self,
        slug: str,
        metric: str,
        from_date: datetime,
        to_date: datetime,
        interval: str = '1d'
    ) -> List[SantimentData]:
        """
        Get metric data for a specific asset
        
        Args:
            slug: Asset slug (e.g., 'bitcoin', 'ethereum')
            metric: Metric name from self.metrics
            from_date: Start date
            to_date: End date
            interval: Time interval ('1h', '1d', '7d')
        """
        
        # Convert metric name to Santiment metric
        santiment_metric = self.metrics.get(metric, metric)
        
        # Format dates
        from_str = from_date.strftime('%Y-%m-%dT%H:%M:%SZ')
        to_str = to_date.strftime('%Y-%m-%dT%H:%M:%SZ')
        
        query = f"""
        {{
            getMetric(metric: "{santiment_metric}") {{
                timeseriesData(
                    slug: "{slug}"
                    from: "{from_str}"
                    to: "{to_str}"
                    interval: "{interval}"
                ) {{
                    datetime
                    value
                }}
            }}
        }}
        """
        
        try:
            result = self.query_graphql(query)
            
            if 'errors' in result:
                logger.error(f"GraphQL errors: {result['errors']}")
                return []
            
            data_points = result.get('data', {}).get('getMetric', {}).get('timeseriesData', [])
            
            santiment_data = []
            for point in data_points:
                if point['value'] is not None:
                    santiment_data.append(SantimentData(
                        slug=slug,
                        metric=metric,
                        datetime=datetime.fromisoformat(point['datetime'].replace('Z', '+00:00')),
                        value=point['value'],
                        metadata={'santiment_metric': santiment_metric, 'interval': interval}
                    ))
            
            logger.info(f"Retrieved {len(santiment_data)} data points for {slug} {metric}")
            return santiment_data
            
        except Exception as e:
            logger.error(f"Error retrieving {metric} for {slug}: {e}")
            return []
    
    def get_social_volume(
        self,
        slug: str,
        from_date: datetime,
        to_date: datetime,
        interval: str = '1d'
    ) -> List[SantimentData]:
        """Get social volume data"""
        return self.get_metric_data(slug, 'social_volume', from_date, to_date, interval)
    
    def get_weighted_sentiment(
        self,
        slug: str,
        from_date: datetime,
        to_date: datetime,
        interval: str = '1d'
    ) -> List[SantimentData]:
        """Get weighted sentiment data"""
        return self.get_metric_data(slug, 'weighted_sentiment', from_date, to_date, interval)
    
    def get_exchange_flows(
        self,
        slug: str,
        from_date: datetime,
        to_date: datetime,
        interval: str = '1d'
    ) -> Dict[str, List[SantimentData]]:
        """Get exchange inflow and outflow data"""
        inflow = self.get_metric_data(slug, 'exchange_inflow', from_date, to_date, interval)
        outflow = self.get_metric_data(slug, 'exchange_outflow', from_date, to_date, interval)
        
        return {
            'inflow': inflow,
            'outflow': outflow
        }
    
    def get_address_metrics(
        self,
        slug: str,
        from_date: datetime,
        to_date: datetime,
        interval: str = '1d'
    ) -> Dict[str, List[SantimentData]]:
        """Get address usage metrics (active addresses)"""
        active = self.get_metric_data(slug, 'active_addresses', from_date, to_date, interval)
        
        return {
            'active_addresses': active
        }
    
    def get_mvrv(
        self,
        slug: str,
        from_date: datetime,
        to_date: datetime,
        interval: str = '1d'
    ) -> List[SantimentData]:
        """Get MVRV (Market Value to Realized Value) data"""
        return self.get_metric_data(slug, 'mvrv_usd', from_date, to_date, interval)
    
    def get_price(
        self,
        slug: str,
        from_date: datetime,
        to_date: datetime,
        interval: str = '1d'
    ) -> List[SantimentData]:
        """Get price in USD"""
        return self.get_metric_data(slug, 'price_usd', from_date, to_date, interval)
    
    def get_npl(
        self,
        slug: str,
        from_date: datetime,
        to_date: datetime,
        interval: str = '1d'
    ) -> List[SantimentData]:
        """Get Network Profit/Loss data"""
        return self.get_metric_data(slug, 'npl', from_date, to_date, interval)
    
    def get_all_metrics(
        self,
        slug: str,
        from_date: datetime,
        to_date: datetime,
        interval: str = '1d'
    ) -> Dict[str, List[SantimentData]]:
        """
        Get all available metrics for an asset
        
        Returns:
            Dictionary with metric names as keys and data lists as values
        """
        logger.info(f"Fetching all metrics for {slug} from {from_date} to {to_date}")
        
        all_data = {}
        
        # Social metrics
        all_data['social_volume'] = self.get_social_volume(slug, from_date, to_date, interval)
        all_data['weighted_sentiment'] = self.get_weighted_sentiment(slug, from_date, to_date, interval)
        
        # Exchange flow metrics
        exchange_flows = self.get_exchange_flows(slug, from_date, to_date, interval)
        all_data['exchange_inflow'] = exchange_flows['inflow']
        all_data['exchange_outflow'] = exchange_flows['outflow']
        
        # Address metrics
        address_metrics = self.get_address_metrics(slug, from_date, to_date, interval)
        all_data['active_addresses'] = address_metrics['active_addresses']
        
        # Financial metrics
        all_data['mvrv_usd'] = self.get_mvrv(slug, from_date, to_date, interval)
        all_data['npl'] = self.get_npl(slug, from_date, to_date, interval)
        
        # Summary
        total_points = sum(len(data) for data in all_data.values())
        logger.info(f"Retrieved {total_points} total data points across {len(all_data)} metrics")
        
        return all_data
    
    def to_dataframe(self, data: List[SantimentData]) -> pd.DataFrame:
        """Convert SantimentData list to pandas DataFrame"""
        if not data:
            return pd.DataFrame()
        
        df_data = []
        for item in data:
            df_data.append({
                'slug': item.slug,
                'metric': item.metric,
                'datetime': item.datetime,
                'value': item.value,
                'santiment_metric': item.metadata.get('santiment_metric'),
                'interval': item.metadata.get('interval')
            })
        
        df = pd.DataFrame(df_data)
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.set_index('datetime')
        
        return df
    
    def get_available_assets(self) -> List[str]:
        """Get list of available asset slugs"""
        query = """
        {
            allProjects {
                slug
                name
                ticker
            }
        }
        """
        
        try:
            result = self.query_graphql(query)
            projects = result.get('data', {}).get('allProjects', [])
            
            asset_list = []
            for project in projects:
                asset_list.append({
                    'slug': project['slug'],
                    'name': project['name'],
                    'ticker': project['ticker']
                })
            
            logger.info(f"Found {len(asset_list)} available assets")
            return asset_list
            
        except Exception as e:
            logger.error(f"Error retrieving available assets: {e}")
            return []


In [10]:
# Initialize client
client = SantimentClient(API_KEY)

# Time range
to_date = datetime.utcnow() - timedelta(days=60)     # 2 months ago
from_date = datetime.utcnow() - timedelta(days=240)  # 8 months ago

slug = "bitcoin"

print("📡 Fetching Bitcoin metrics...")

# Fetch price (via market_cap_usd)
price_data = client.get_market_cap_usd(slug, from_date, to_date, interval="1d")
price_df = client.to_dataframe(price_data)[["value"]].rename(columns={"value": "price"})

metric_dfs = {"price": price_df}

# Fetch all other metrics available in the client
for metric in client.metrics.keys():
    if metric == "market_cap_usd":
        continue  # Already used as price

    print(f" → Fetching {metric} ...")
    data = client.get_metric_data(slug, metric, from_date, to_date, interval="1d")
    df = client.to_dataframe(data)
    print(f"   Returned shape for metric {metric}: {df.shape}")

    if df.empty:
        print(f"   ⚠️ No data returned for {metric}")
        continue

    df = df[["value"]].rename(columns={"value": metric})
    metric_dfs[metric] = df

# Merge all metrics on datetime
print("\n🔄 Merging all time series...")
merged_df = pd.concat(metric_dfs.values(), axis=1, join="inner")

print(f"📊 Final dataset shape: {merged_df.shape}")

# -----------------------------
# FULL CORRELATION MATRIX
# -----------------------------
full_corr_matrix = merged_df.corr()

# -----------------------------
# CORRELATION vs PRICE (sorted)
# -----------------------------
corr_vs_price = full_corr_matrix[["price"]].sort_values("price", ascending=False)

print("\n===== 📈 Correlation vs Bitcoin Price (sorted) =====\n")
display(corr_vs_price)

print("\n===== 🔍 Full Correlation Matrix (all variables vs each other) =====\n")
display(full_corr_matrix)


📡 Fetching Bitcoin metrics...


C:\Users\pavau\AppData\Local\Temp\ipykernel_32512\305974814.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  to_date = datetime.utcnow() - timedelta(days=60)     # 2 months ago
C:\Users\pavau\AppData\Local\Temp\ipykernel_32512\305974814.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  from_date = datetime.utcnow() - timedelta(days=240)  # 8 months ago


AttributeError: 'SantimentClient' object has no attribute 'get_market_cap_usd'

In [11]:
# 🧪 TEST: Single API Call Rate Limit Analysis
print("🧪 Single API Call Rate Limit Test")
print("=" * 80)

client = SantimentClient(API_KEY)

# Test configuration
test_assets = ['bitcoin', 'ethereum', 'cardano', 'solana', 'chainlink', 'uniswap', 'polygon', 'avalanche', 'polkadot', 'litecoin', 'bitcoin-cash', 'stellar', 'algorand', 'cosmos', 'tezos', 'filecoin', 'aave', 'compound', 'maker', 'dash']  # Twenty assets
test_assets.extend(['monero', 'zcash', 'eos', 'tron', 'neo', 'vechain', 'theta', 'fantom', 'near', 'aptos'])  # Add 10 more assets
test_assets.extend(['sui', 'arbitrum', 'optimism', 'base', 'celestia', 'injective', 'sei', 'sei-network', 'render-token', 'fetch-ai'])  # Add 10 more assets
test_metrics = ['active_addresses', 'price_usd', 'marketcap_usd', 'exchange_inflow', 'exchange_outflow', 'social_volume', 'weighted_sentiment', 'npl']  # Nine metrics for each asset (all available)
test_start = datetime.now() - timedelta(days=90)
test_end = datetime.now() - timedelta(days=60)

print(f"\n📋 Test Configuration:")
print(f"   Assets: {', '.join([asset.title() for asset in test_assets])}")
print(f"   Metrics: {', '.join([metric.replace('_', ' ').title() for metric in test_metrics])}")
print(f"   Total queries: {len(test_assets)} assets × {len(test_metrics)} metrics = {len(test_assets) * len(test_metrics)} getMetric calls")
print(f"   Date range: {test_start.date()} to {test_end.date()}")

# Get rate limit BEFORE making the call - ensure we have enough time before window reset
print(f"\n📊 Rate Limit Status BEFORE API Call:")

# Make a test call to inspect ALL headers
print(f"\n{'='*80}")
print("📋 INSPECTING API RESPONSE HEADERS")
print(f"{'='*80}")
test_query = '{ getMetric(metric: "active_addresses_24h") { timeseriesData(slug: "bitcoin", from: "2025-01-01T00:00:00Z", to: "2025-01-02T00:00:00Z", interval: "1d") { datetime value } } }'
test_response = client.session.post(client.base_url, json={'query': test_query}, timeout=30)
test_response.raise_for_status()
# Update rate limit info from headers
client._update_rate_limit_info(test_response.headers)

# Remove header printing - we know they're there, just parse them

# Make a test call to get headers BEFORE
print(f"\n{'='*80}")
print("📊 HEADERS BEFORE API CALL")
print(f"{'='*80}")
test_query = '{ getMetric(metric: "active_addresses_24h") { timeseriesData(slug: "bitcoin", from: "2025-01-01T00:00:00Z", to: "2025-01-02T00:00:00Z", interval: "1d") { datetime value } } }'
response_before = client.session.post(client.base_url, json={'query': test_query}, timeout=30)
response_before.raise_for_status()

print("\nResponse Headers BEFORE:")
for header, value in response_before.headers.items():
    print(f"   {header}: {value}")

print(f"\n{'='*80}")
print("🔄 Making Single API Call...")
print(f"{'='*80}")

# Make the API call - Single call for two assets and two metrics using GraphQL aliases
try:
    # Create GraphQL query with aliases to query both assets and both metrics in one call
    from_str = test_start.strftime('%Y-%m-%dT%H:%M:%SZ')
    to_str = test_end.strftime('%Y-%m-%dT%H:%M:%SZ')
    
    # Build query with aliases for each asset-metric combination
    query_parts = []
    for asset in test_assets:
        for metric in test_metrics:
            santiment_metric = client.metrics.get(metric, metric)
            alias = f"{asset.replace('-', '_')}_{metric.replace('-', '_')}"
            query_parts.append(f"""
            {alias}: getMetric(metric: "{santiment_metric}") {{
                timeseriesData(
                    slug: "{asset}"
                    from: "{from_str}"
                    to: "{to_str}"
                    interval: "5m"
                ) {{
                    datetime
                    value
                }}
            }}
        """)
    
    query = "{" + "".join(query_parts) + "}"
    
    # Execute the API call and capture response
    result = client.query_graphql(query)
    
    # Get headers AFTER the call
    print(f"\n{'='*80}")
    print("📊 HEADERS AFTER API CALL")
    print(f"{'='*80}")
    
    # Access the last response from the client
    if hasattr(client, 'last_response') and client.last_response:
        response_after = client.last_response
        print("\nResponse Headers AFTER:")
        for header, value in response_after.headers.items():
            print(f"   {header}: {value}")
    else:
        # Make another call to get headers after
        test_query_after = '{ getMetric(metric: "active_addresses_24h") { timeseriesData(slug: "bitcoin", from: "2025-01-01T00:00:00Z", to: "2025-01-02T00:00:00Z", interval: "1d") { datetime value } } }'
        response_after = client.session.post(client.base_url, json={'query': test_query_after}, timeout=30)
        response_after.raise_for_status()
        print("\nResponse Headers AFTER:")
        for header, value in response_after.headers.items():
            print(f"   {header}: {value}")
    
except Exception as e:
    print(f"   ❌ Error making API call: {str(e)[:100]}...")
    import traceback
    traceback.print_exc()

print(f"\n✅ Single API call test complete!")

🧪 Single API Call Rate Limit Test

📋 Test Configuration:
   Assets: Bitcoin, Ethereum, Cardano, Solana, Chainlink, Uniswap, Polygon, Avalanche, Polkadot, Litecoin, Bitcoin-Cash, Stellar, Algorand, Cosmos, Tezos, Filecoin, Aave, Compound, Maker, Dash, Monero, Zcash, Eos, Tron, Neo, Vechain, Theta, Fantom, Near, Aptos, Sui, Arbitrum, Optimism, Base, Celestia, Injective, Sei, Sei-Network, Render-Token, Fetch-Ai
   Metrics: Active Addresses, Price Usd, Marketcap Usd, Exchange Inflow, Exchange Outflow, Social Volume, Weighted Sentiment, Npl
   Total queries: 40 assets × 8 metrics = 320 getMetric calls
   Date range: 2025-09-14 to 2025-10-14

📊 Rate Limit Status BEFORE API Call:

📋 INSPECTING API RESPONSE HEADERS


ConnectionError: ('Connection aborted.', PermissionError(13, 'Permission denied'))

In [9]:
"""
Get list of all tickers and slugs from Santiment API
"""

client = SantimentClient(API_KEY)

import pandas as pd

# Query for all assets with tickers, slugs, and market segment
tickers_slugs_query = """
{
  allProjects {
    ticker
    slug
    marketSegment
  }
}
"""

try:
    result = client.query_graphql(tickers_slugs_query)
    projects = result.get('data', {}).get('allProjects', [])
    
    # Create DataFrame with ticker, slug, and market segment
    df = pd.DataFrame(projects)
    
    # Sort by ticker for easier reading
    df = df.sort_values('ticker').reset_index(drop=True)
    
    print("=" * 80)
    print("📋 Tickers, Slugs, and Market Segments")
    print("=" * 80)
    print(f"\n✅ Found {len(df)} assets\n")
    print(df.to_string(index=False))
    
    # Also create a simple mapping dictionary
    ticker_to_slug = {row['ticker']: row['slug'] for _, row in df.iterrows() if row['ticker']}
    slug_to_ticker = {row['slug']: row['ticker'] for _, row in df.iterrows() if row['slug']}
    
    print(f"\n💡 Variables created:")
    print(f"   - df: DataFrame with ticker, slug, and marketSegment columns")
    print(f"   - ticker_to_slug: Dictionary mapping ticker → slug")
    print(f"   - slug_to_ticker: Dictionary mapping slug → ticker")
    
except Exception as e:
    print(f"❌ Error fetching tickers and slugs: {e}")
    import traceback
    traceback.print_exc()


ERROR:__main__:GraphQL query failed: ('Connection aborted.', PermissionError(13, 'Permission denied'))


❌ Error fetching tickers and slugs: ('Connection aborted.', PermissionError(13, 'Permission denied'))


Traceback (most recent call last):
  File "c:\Users\pavau\anaconda3\envs\Proofofconcept\Lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_request(
        conn,
    ...<10 lines>...
        **response_kw,
    )
  File "c:\Users\pavau\anaconda3\envs\Proofofconcept\Lib\site-packages\urllib3\connectionpool.py", line 488, in _make_request
    raise new_e
  File "c:\Users\pavau\anaconda3\envs\Proofofconcept\Lib\site-packages\urllib3\connectionpool.py", line 464, in _make_request
    self._validate_conn(conn)
    ~~~~~~~~~~~~~~~~~~~^^^^^^
  File "c:\Users\pavau\anaconda3\envs\Proofofconcept\Lib\site-packages\urllib3\connectionpool.py", line 1093, in _validate_conn
    conn.connect()
    ~~~~~~~~~~~~^^
  File "c:\Users\pavau\anaconda3\envs\Proofofconcept\Lib\site-packages\urllib3\connection.py", line 790, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
        sock=sock,
    ...<14 lines>...
        assert_fingerprint=self.ass

In [ ]:
"""
Check for duplicate tickers and see what fields are available to distinguish them
"""

import pandas as pd
from collections import Counter

# Query for all assets with additional fields that might help distinguish them
detailed_query = """
{
  allProjects {
    slug
    name
    ticker
    id
    infrastructure
    marketSegment
  }
}
"""

try:
    result = client.query_graphql(detailed_query)
    projects = result.get('data', {}).get('allProjects', [])
    
    df = pd.DataFrame(projects)
    
    # Find duplicate tickers
    ticker_counts = Counter(df['ticker'].dropna())
    duplicates = {ticker: count for ticker, count in ticker_counts.items() if count > 1}
    
    print("=" * 80)
    print("🔍 Duplicate Tickers Analysis")
    print("=" * 80)
    print(f"\n✅ Found {len(duplicates)} tickers with duplicates ({sum(duplicates.values())} total assets)\n")
    
    if duplicates:
        print("Duplicate tickers:")
        for ticker, count in sorted(duplicates.items(), key=lambda x: x[1], reverse=True)[:20]:
            print(f"   {ticker}: {count} assets")
        
        # Show examples of duplicate tickers
        print("\n" + "=" * 80)
        print("📋 Example: Assets with ticker 'DOGE'")
        print("=" * 80)
        doge_assets = df[df['ticker'] == 'DOGE'][['slug', 'name', 'ticker', 'id', 'infrastructure', 'marketSegment']]
        print(doge_assets.to_string(index=False))
        
        # Show what fields are available and non-null
        print("\n" + "=" * 80)
        print("📊 Field Availability for Duplicate Tickers")
        print("=" * 80)
        duplicate_df = df[df['ticker'].isin(duplicates.keys())]
        print("\nNon-null field counts:")
        for col in ['name', 'id', 'infrastructure', 'marketSegment']:
            non_null = duplicate_df[col].notna().sum()
            print(f"   {col}: {non_null}/{len(duplicate_df)} ({100*non_null/len(duplicate_df):.1f}%)")
    
    # Save the detailed dataframe
    detailed_assets_df = df
    
    print("\n💡 Key points:")
    print("   - Use 'slug' as the unique identifier (required for API queries)")
    print("   - 'name' field can help distinguish assets with same ticker")
    print("   - 'id' is a unique numeric identifier for each asset")
    print("   - 'infrastructure' field shows the blockchain/platform (e.g., 'ethereum', 'bitcoin')")
    print("   - 'marketSegment' provides additional categorization")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


🔍 Duplicate Tickers Analysis

✅ Found 282 tickers with duplicates (631 total assets)

Duplicate tickers:
   FRAX: 7 assets
   USDT: 7 assets
   USDC: 7 assets
   DAI: 6 assets
   MIM: 5 assets
   BUSD: 4 assets
   TRUMP: 4 assets
   SHIDO: 4 assets
   BLOCK: 4 assets
   WBTC: 4 assets
   WETH: 4 assets
   BROCCOLI: 4 assets
   MAGA: 3 assets
   RAIN: 3 assets
   BAL: 3 assets
   SPX: 3 assets
   NEIRO: 3 assets
   NCT: 3 assets
   BIFI: 3 assets
   CAT: 3 assets

📋 Example: Assets with ticker 'DOGE'
                                     slug                                              name ticker     id infrastructure  marketSegment
                                 dogecoin                                          Dogecoin   DOGE 234314       Dogecoin Cryptocurrency
department-of-government-efficiency-token Department Of Government Efficiency (dogegov.com)   DOGE 238996            ETH       Ethereum

📊 Field Availability for Duplicate Tickers

Non-null field counts:
   name: 631/631 (1

In [ ]:
"""
Helper functions for working with duplicate tickers
"""

def get_slugs_by_ticker(ticker: str, df: pd.DataFrame = None) -> pd.DataFrame:
    """
    Get all slugs for a given ticker
    
    Args:
        ticker: Ticker symbol (e.g., 'DOGE', 'USDT')
        df: DataFrame from detailed_assets_df (if None, uses the one from previous cell)
    
    Returns:
        DataFrame with all assets matching the ticker
    """
    if df is None:
        try:
            df = detailed_assets_df
        except NameError:
            print("❌ detailed_assets_df not found. Run the duplicate tickers analysis cell first.")
            return pd.DataFrame()
    
    matches = df[df['ticker'] == ticker.upper()].copy()
    return matches[['slug', 'name', 'ticker', 'id', 'infrastructure', 'marketSegment']]


def get_slug_by_ticker_and_infrastructure(ticker: str, infrastructure: str = None, df: pd.DataFrame = None) -> str:
    """
    Get slug for a ticker, optionally filtered by infrastructure
    
    Args:
        ticker: Ticker symbol
        infrastructure: Optional infrastructure filter (e.g., 'ETH', 'Dogecoin')
        df: DataFrame from detailed_assets_df
    
    Returns:
        Slug string, or None if not found/ambiguous
    """
    matches = get_slugs_by_ticker(ticker, df)
    
    if len(matches) == 0:
        print(f"❌ No assets found with ticker '{ticker}'")
        return None
    
    if infrastructure:
        matches = matches[matches['infrastructure'].str.contains(infrastructure, case=False, na=False)]
        if len(matches) == 0:
            print(f"❌ No assets found with ticker '{ticker}' and infrastructure '{infrastructure}'")
            return None
    
    if len(matches) == 1:
        return matches.iloc[0]['slug']
    else:
        print(f"⚠️  Multiple matches found for '{ticker}':")
        print(matches[['slug', 'name', 'infrastructure']].to_string(index=False))
        print(f"\n💡 Use get_slugs_by_ticker('{ticker}') to see all options")
        return None


def filter_duplicates_by_infrastructure(df: pd.DataFrame = None) -> pd.DataFrame:
    """
    Show duplicate tickers grouped by infrastructure
    
    Args:
        df: DataFrame from detailed_assets_df
    
    Returns:
        DataFrame showing duplicates grouped by ticker and infrastructure
    """
    if df is None:
        try:
            df = detailed_assets_df
        except NameError:
            print("❌ detailed_assets_df not found. Run the duplicate tickers analysis cell first.")
            return pd.DataFrame()
    
    # Get duplicates
    ticker_counts = df['ticker'].value_counts()
    duplicate_tickers = ticker_counts[ticker_counts > 1].index
    
    # Filter and group
    duplicates_df = df[df['ticker'].isin(duplicate_tickers)].copy()
    duplicates_df = duplicates_df.sort_values(['ticker', 'infrastructure', 'name'])
    
    print("=" * 80)
    print("📊 Duplicate Tickers by Infrastructure")
    print("=" * 80)
    
    for ticker in sorted(duplicate_tickers)[:10]:  # Show first 10
        ticker_assets = duplicates_df[duplicates_df['ticker'] == ticker]
        print(f"\n{ticker} ({len(ticker_assets)} assets):")
        for _, row in ticker_assets.iterrows():
            infra = row['infrastructure'] if pd.notna(row['infrastructure']) else 'N/A'
            print(f"   - {row['name']} (slug: {row['slug']}, infrastructure: {infra})")
    
    if len(duplicate_tickers) > 10:
        print(f"\n... and {len(duplicate_tickers) - 10} more duplicate tickers")
    
    return duplicates_df


# Example usage:
print("✅ Helper functions loaded!")
print("\nAvailable functions:")
print("   - get_slugs_by_ticker('DOGE') - Get all DOGE assets")
print("   - get_slug_by_ticker_and_infrastructure('DOGE', 'ETH') - Get ETH-based DOGE slug")
print("   - filter_duplicates_by_infrastructure() - Show duplicates grouped by infrastructure")
print("\nExample:")
print("   doge_slugs = get_slugs_by_ticker('DOGE')")
print("   eth_doge_slug = get_slug_by_ticker_and_infrastructure('DOGE', 'ETH')")
